In [1]:
import pickle
import sentencepiece as spm
import json
from glob import glob
import os
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tensor2tensor.layers import modalities
import tensorflow as tf
from tqdm import tqdm

In [2]:
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)


class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100

    def encode(self, s):
        return self.sp.EncodeAsIds(s)

    def decode(self, ids, strip_extraneous = False):
        return self.sp.DecodeIds(list(ids))

In [3]:
d = [
    {'class': 0, 'Description': 'PAD', 'salah': '', 'betul': ''},
    {
        'class': 1,
        'Description': 'kesambungan subwords',
        'salah': '',
        'betul': '',
    },
    {'class': 2, 'Description': 'tiada kesalahan', 'salah': '', 'betul': ''},
    {
        'class': 3,
        'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
        'salah': 'Cili sos',
        'betul': 'sos cili',
    },
    {
        'class': 4,
        'Description': 'kesalahan kata jamak',
        'salah': 'mereka-mereka',
        'betul': 'mereka',
    },
    {
        'class': 5,
        'Description': 'kesalahan kata penguat',
        'salah': 'sangat tinggi sekali',
        'betul': 'sangat tinggi',
    },
    {
        'class': 6,
        'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat.',
        'salah': 'Sani mendapat markah yang tertinggi sekali.',
        'betul': 'Sani mendapat markah yang tertinggi.',
    },
    {
        'class': 7,
        'Description': 'kesalahan kata hubung',
        'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
        'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
    },
    {
        'class': 8,
        'Description': 'kesalahan kata bilangan',
        'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
        'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
    },
    {
        'class': 9,
        'Description': 'kesalahan kata sendi',
        'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
        'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
    },
    {
        'class': 10,
        'Description': 'kesalahan penjodoh bilangan',
        'salah': 'Setiap orang pelajar',
        'betul': 'Setiap pelajar.',
    },
    {
        'class': 11,
        'Description': 'kesalahan kata ganti diri',
        'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
        'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
    },
    {
        'class': 12,
        'Description': 'kesalahan ayat pasif',
        'salah': 'Cerpen itu telah dikarang oleh saya.',
        'betul': 'Cerpen itu telah saya karang.',
    },
    {
        'class': 13,
        'Description': 'kesalahan kata tanya',
        'salah': 'Kamu berasal dari manakah ?',
        'betul': 'Kamu berasal dari mana ?',
    },
    {
        'class': 14,
        'Description': 'kesalahan tanda baca',
        'salah': 'Kamu berasal dari manakah .',
        'betul': 'Kamu berasal dari mana ?',
    },
    {
        'class': 15,
        'Description': 'kesalahan kata kerja tak transitif',
        'salah': 'Dia kata kepada saya',
        'betul': 'Dia berkata kepada saya',
    },
    {
        'class': 16,
        'Description': 'kesalahan kata kerja tak transitif',
        'salah': 'Dia kata kepada saya',
        'betul': 'Dia berkata kepada saya',
    },
    {
        'class': 17,
        'Description': 'kesalahan kata kerja transitif',
        'salah': 'Dia suka baca buku',
        'betul': 'Dia suka membaca buku',
    },
    {
        'class': 18,
        'Description': 'penggunaan kata yang tidak tepat',
        'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
        'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
    },
    {
        'class': 19,
        'Description': 'kesalahan frasa kerja tak transitif',
        'salah': 'berdasarkan pada keterangan ini',
        'betul': 'berdasarkan keterangan ini',
    },
    {
        'class': 20,
        'Description': 'kesalahan frasa kerja transitif',
        'salah': 'Dia membeli banyak buah',
        'betul': 'Dia banyak membeli buah',
    },
    {
        'class': 21,
        'Description': 'kesalahan frasa kerja pasif',
        'salah': 'Surat itu saga akan balas',
        'betul': 'Surat itu akan saga balas',
    },
]


class Tatabahasa:
    def __init__(self, d):
        self.d = d
        self.kesalahan = {i['Description']: no for no, i in enumerate(self.d)}
        self.reverse_kesalahan = {v: k for k, v in self.kesalahan.items()}
        self.vocab_size = len(self.d)

    def encode(self, s):
        return [self.kesalahan[i] for i in s]

    def decode(self, ids, strip_extraneous = False):
        return [self.reverse_kesalahan[i] for i in ids]

In [4]:
@registry.register_problem
class Grammar(text_problems.Text2TextProblem):
    """grammatical error correction."""

    def feature_encoders(self, data_dir):
        encoder = Encoder(sp)
        t = Tatabahasa(d)
        return {'inputs': encoder, 'targets': encoder, 'targets_error_tag': t}

    def hparams(self, defaults, model_hparams):
        super(Grammar, self).hparams(defaults, model_hparams)
        if 'use_error_tags' not in model_hparams:
            model_hparams.add_hparam('use_error_tags', True)
        if 'middle_prediction' not in model_hparams:
            model_hparams.add_hparam('middle_prediction', False)
        if 'middle_prediction_layer_factor' not in model_hparams:
            model_hparams.add_hparam('middle_prediction_layer_factor', 2)
        if 'ffn_in_prediction_cascade' not in model_hparams:
            model_hparams.add_hparam('ffn_in_prediction_cascade', 1)
        if 'error_tag_embed_size' not in model_hparams:
            model_hparams.add_hparam('error_tag_embed_size', 12)
        if model_hparams.use_error_tags:
            defaults.modality[
                'targets_error_tag'
            ] = modalities.ModalityType.SYMBOL
            error_tag_vocab_size = self._encoders[
                'targets_error_tag'
            ].vocab_size
            defaults.vocab_size['targets_error_tag'] = error_tag_vocab_size

    def example_reading_spec(self):
        data_fields, _ = super(Seq2edits, self).example_reading_spec()
        data_fields['targets_error_tag'] = tf.VarLenFeature(tf.int64)
        return data_fields, None

    @property
    def approx_vocab_size(self):
        return 32100

    @property
    def is_generate_per_split(self):
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 200},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        with open('../pure-text/dataset-tatabahasa.pkl', 'rb') as fopen:
            data = pickle.load(fopen)

        encoder = Encoder(sp)
        for row in tqdm(data):
            x, y, tag = get_xy(row, encoder)
            yield {
                'inputs': x,
                'targets': y,
                'targets_error_tag': tag,
            }

    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):

        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in generator:
            yield sample

In [5]:
with open('../pure-text/dataset-tatabahasa.pkl', 'rb') as fopen:
    data = pickle.load(fopen)

encoder = Encoder(sp)

In [6]:
def get_xy(row, encoder):
    x, y, tag = [], [], []

    for i in range(len(row[0])):
        t = encoder.encode(row[0][i][0])
        y.extend(t)
        t = encoder.encode(row[1][i][0])
        x.extend(t)
        tag.extend([row[1][i][1]] * len(t))

    # EOS
    x.append(1)
    y.append(1)
    tag.append(0)

    return x, y, tag

In [10]:
x, y, tag = get_xy(data[0], encoder)
x, y, tag

([104,
  6892,
  3208,
  11382,
  13,
  13,
  25,
  15,
  6,
  11382,
  13,
  13,
  25,
  7,
  749,
  36,
  15,
  6,
  15277,
  844,
  13,
  564,
  15,
  4,
  2083,
  417,
  727,
  4073,
  15,
  5,
  34,
  648,
  714,
  1337,
  394,
  17,
  798,
  18,
  4963,
  3481,
  15,
  3,
  1],
 [104,
  6892,
  3208,
  11382,
  13,
  13,
  25,
  15,
  6,
  11382,
  13,
  13,
  25,
  7,
  749,
  36,
  15,
  6,
  15277,
  844,
  13,
  564,
  15,
  4,
  2083,
  417,
  727,
  4073,
  15,
  5,
  34,
  394,
  648,
  714,
  1337,
  17,
  798,
  18,
  3481,
  4963,
  15,
  3,
  1],
 [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  2,
  2,
  2,
  3,
  3,
  2,
  2,
  0])

In [11]:
encoder.decode(x)

'Dirk Jan Klaas " Klaas-Jan " Huntelaar ( lahir 12 Ogos 1983 ) merupakan bola sepak Belanda pemain yang bermain di penyerang posisi .'

In [12]:
encoder.decode(y)

'Dirk Jan Klaas " Klaas-Jan " Huntelaar ( lahir 12 Ogos 1983 ) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang .'

In [13]:
import os
import tensorflow as tf

os.system('rm -rf t2t-tatabahasa/data')
DATA_DIR = os.path.expanduser('t2t-tatabahasa/data')
TMP_DIR = os.path.expanduser('t2t-tatabahasa/tmp')

In [14]:
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [15]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

In [16]:
PROBLEM = 'grammar'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
100%|██████████| 5000/5000 [00:02<00:00, 2203.33it/s]

INFO:tensorflow:Generated 5000 Examples



INFO:tensorflow:Generated 5000 Examples


...

INFO:tensorflow:Shuffling data...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.
